In [ ]:
%cd ../..

In [ ]:
saved = "v31_results/" # "ml_hep_sim/analysis/results/ul/"

# Get hists pipeline

In [ ]:
from ml_hep_sim.analysis.hists_pipeline import get_hists_pipeline

In [ ]:
use_class = False

hists_pipeline = get_hists_pipeline(use_classifier=use_class)
hists_pipeline.pipes = hists_pipeline.pipes[:-1] # this one only makes 1 hist

In [ ]:
if use_class:
    saved += "class_"
else:
    saved += "mbb_"

In [ ]:
# import logging

# logger = logging.getLogger()
# logger.disabled = True

In [ ]:
saved

# Make many hists

In [ ]:
from ml_hep_sim.analysis.hists_pipeline import MakeHistsFromSamplesLumi
from ml_hep_sim.analysis.ul_pipeline import UpperLimitScannerBlock
import numpy as np

In [ ]:
lumi_start, lumi_end, lumi_step = 100, 1000, 24
lumi = [lumi_start, lumi_end, lumi_step]

sig_frac = 0.1
xsec = 10

## Lumi table

In [ ]:
_sig, _bkg = [], []

for l in np.linspace(*lumi):
    _sig.append(l * xsec * sig_frac)
    _bkg.append(l * xsec)
    print(f"lumi: {l} sig: {_sig[-1]}, bkg: {_bkg[-1]}")

# UL setup

In [ ]:
b_sig_bkg = hists_pipeline.pipes[-1]

In [ ]:
sig_fracs = np.linspace(0.01, 0.1, 6) # do different signal fraction (injections)

In [ ]:
sig_fracs

In [ ]:
# set mu and gamma (mc correction) bounds
poi_bound = (0.0, 2.0)
gamma_bound = (1e-10, 2.0)

N_gen = 10**6
bkg_err = 0.1 # sys error
bins = 30
bounds = []

for b in range(2*bins+1):
    if b == 0:
        bounds.append(poi_bound)
    else:
        bounds.append(gamma_bound)

In [ ]:
if use_class:
    bin_range = (0.5, 1.1)
else:
    bin_range = (0.01, 3.0)

hists_ul_blocks = []

for sf in sig_fracs:
    hists_block = MakeHistsFromSamplesLumi(bin_range=bin_range, N_gen=N_gen, 
                                           bins=bins, alpha=sf, lumi=lumi, xsec=xsec)(b_sig_bkg)
    
    ul_block = UpperLimitScannerBlock(bkg_err=bkg_err, workers=24, par_bounds=bounds, mc_test=False)(hists_block)

    hists_ul_blocks.append(hists_block)
    hists_ul_blocks.append(ul_block)

for sf in sig_fracs:
    hists_block = MakeHistsFromSamplesLumi(bin_range=bin_range, N_gen=N_gen,
                                           bins=bins, alpha=sf, lumi=lumi, xsec=xsec)(b_sig_bkg)
    ul_block = UpperLimitScannerBlock(bkg_err=bkg_err, workers=24, par_bounds=bounds, mc_test=True)(hists_block)

    hists_ul_blocks.append(hists_block)
    hists_ul_blocks.append(ul_block)

# Build pipeline

In [ ]:
from ml_hep_sim.pipeline.pipes import Pipeline

pipe = Pipeline()
pipe.compose(hists_pipeline, hists_ul_blocks)
pipe.fit()

In [ ]:
# idx for results from pipeline
idxs_gen = [-1, -3, -5, -7, -9, -11]
idxs_mc = [-13, -15, -17, -19, -21, -23]

In [ ]:
sig_fracs

# Plot UL

In [ ]:
import matplotlib.pyplot as plt
from ml_hep_sim.plotting.style import style_setup, set_size

set_size()
style_setup(seaborn_pallete=True)

In [ ]:
for i, (idx_gen, idx_mc) in enumerate(zip(idxs_gen, idxs_mc)):

    res_gen = pipe.pipes[idx_gen].results
    res_mc = pipe.pipes[idx_mc].results

    plt.fill_between(res_mc.N, res_mc.minus_sigma_2, res_mc.plus_sigma_2, color="yellow", label="$\pm 2\sigma$")
    plt.fill_between(res_mc.N, res_mc.minus_sigma_1, res_mc.plus_sigma_1, color="green", label="$\pm 1\sigma$")

    plt.plot(res_mc.N, res_mc.minus_sigma_1, c='k', ls='dotted')
    plt.plot(res_mc.N, res_mc.minus_sigma_2, c='k', ls='dotted')

    plt.plot(res_mc.N, res_mc.plus_sigma_1, c='k', ls='dotted')
    plt.plot(res_mc.N, res_mc.plus_sigma_2, c='k', ls='dotted')

    plt.plot(res_mc.N, res_mc.cls_obs, zorder=20, color='k', label="$\mu$ MC obs")
    plt.plot(res_mc.N, res_mc.cls_exp, color='k', ls='--', label="$\mu$ exp")

    plt.plot(res_gen.N, res_gen.cls_obs, zorder=20, color='r', label="$\mu$ ML obs")
    
    # plt.plot(res_gen.N, res_gen.cls_exp, color='r', ls='--', label="$\mu$ exp") # off

    # plt.xlabel("$N_{gen}^{bkg}$", loc="center")
    plt.xlabel("$L$ [fb$^{-1}$]")
    plt.ylabel("UL $\mu$")

    # plt.xscale("log")

    plt.title(r"Signal fraction:" + "{:.1f} %".format(sig_fracs[::-1][i] * 100))


    plt.legend()
    plt.tight_layout()
    plt.savefig(saved + f"ul_mu_alpha{i+1}.pdf")
    plt.show()

In [ ]:
for i, (idx_gen, idx_mc) in enumerate(zip(idxs_gen, idxs_mc)):
    res_gen = pipe.pipes[idx_gen].results
    res_mc = pipe.pipes[idx_mc].results

    y = res_mc.cls_obs - res_mc.cls_exp

    plt.fill_between(res_mc.N, res_mc.minus_sigma_2 + y, res_mc.plus_sigma_2 + y, color="yellow", label="$\pm 2\sigma$")
    plt.fill_between(res_mc.N, res_mc.minus_sigma_1 + y, res_mc.plus_sigma_1 + y, color="green", label="$\pm 1\sigma$")

    plt.plot(res_mc.N, res_mc.minus_sigma_1 + y, c='k', ls='dotted')

    plt.plot(res_mc.N, res_mc.plus_sigma_1 + y, c='k', ls='dotted')
    
    plt.plot(res_mc.N, res_mc.minus_sigma_2 + y, c='k', ls='dotted')
    plt.plot(res_mc.N, res_mc.plus_sigma_2 + y, c='k', ls='dotted')

    plt.plot(res_mc.N, res_mc.cls_obs, zorder=20, color='k', label="$\mu$ MC obs")
    # plt.plot(res_mc.N, res_mc.cls_exp, color='k', ls='--', label="$\mu$ exp")

    plt.plot(res_gen.N, res_gen.cls_obs, zorder=20, color='r', label="$\mu$ ML obs")
    
    # plt.xlabel("$N_{gen}^{bkg}$", loc="center")
    plt.xlabel("$L$ [fb$^{-1}$]")
    plt.ylabel("UL $\mu$")

    plt.title(r"Signal fraction:" + "{:.1f} %".format(sig_fracs[::-1][i] * 100))

    plt.legend()
    plt.tight_layout()
    plt.savefig(saved + f"ul_mu_alpha{i+1}_new.pdf")
    plt.show()